In [1]:
import os
import sys
#os.environ["PYSPARK_PYTHON"] = "/opt/continuum/anaconda/bin/python"
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_221.jdk/Contents/Home/jre"
os.environ["SPARK_HOME"] = "/Users/lcx/package/spark-2.4.6-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"

sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.master("local").appName("twitter ML").getOrCreate()

# get the spark context
sc = spark.sparkContext

In [3]:
spark

### 1. Read data from asw folder, text file

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [5]:
files_path = "../twitter_data/sample_data/2020_10_30/*/*"

In [6]:
file_schema = StructType([StructField("tweet_text", StringType(), True), 
                          # StructField("hash_tag", ArrayType(StringType(), True), True), 
                          StructField("hash_tag", StringType(), True), 
                          StructField("created_at", StringType(), True), 
                          StructField("retweet_count", IntegerType(), True), 
                          StructField("favorite_count", IntegerType(), True), 
                          StructField("retweeted", BooleanType(), True), 
                          StructField("truncated", BooleanType(), True), 
                          StructField("id", StringType(), True), 
                          StructField("user_name", StringType(), True), 
                          StructField("screen_name", StringType(), True), 
                          StructField("followers_count", IntegerType(), True), 
                          StructField("location", StringType(), True), 
                          StructField("geo", StringType(), True),
                          StructField("invalid", StringType(), True)])

# file_schema = StructType([StructField("_c0", StringType(), True), 
#                           StructField("_c1", ArrayType(StringType(), True), True), 
#                           StructField("_c2", StringType(), True), 
#                           StructField("_c3", IntegerType(), True), 
#                           StructField("_c4", IntegerType(), True), 
#                           StructField("_c5", BooleanType(), True), 
#                           StructField("_c6", BooleanType(), True), 
#                           StructField("_c7", StringType(), True), 
#                           StructField("_c8", StringType(), True), 
#                           StructField("_c9", StringType(), True), 
#                           StructField("_c10", IntegerType(), True), 
#                           StructField("_c11", StringType(), True), 
#                           StructField("_c12", StringType(), True),
#                           StructField("_c13", StringType(), True)])

In [7]:
dir(file_schema)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_needConversion',
 '_needSerializeAnyField',
 'add',
 'fieldNames',
 'fields',
 'fromInternal',
 'fromJson',
 'json',
 'jsonValue',
 'names',
 'needConversion',
 'simpleString',
 'toInternal',
 'typeName']

In [8]:
len(file_schema.fieldNames())

14

In [9]:
file_schema

StructType(List(StructField(tweet_text,StringType,true),StructField(hash_tag,StringType,true),StructField(created_at,StringType,true),StructField(retweet_count,IntegerType,true),StructField(favorite_count,IntegerType,true),StructField(retweeted,BooleanType,true),StructField(truncated,BooleanType,true),StructField(id,StringType,true),StructField(user_name,StringType,true),StructField(screen_name,StringType,true),StructField(followers_count,IntegerType,true),StructField(location,StringType,true),StructField(geo,StringType,true),StructField(invalid,StringType,true)))

In [7]:
# df = (spark
#       .read
#       .csv(header=False, sep="\t", schema=file_schema, enforceSchema=True, 
#            path=files_path)) # don't know why this does not work

df = (spark
      .read
      .format("csv")
      .options(header=False, sep="\t", enforceSchema=True)
      .schema(file_schema)
      .load(files_path))

In [8]:
df.show(3)

+--------------------+--------------------+--------------------+-------------+--------------+---------+---------+-------------------+--------------------+---------------+---------------+--------------------+----+-------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|retweeted|truncated|                 id|           user_name|    screen_name|followers_count|            location| geo|invalid|
+--------------------+--------------------+--------------------+-------------+--------------+---------+---------+-------------------+--------------------+---------------+---------------+--------------------+----+-------+
|.@mit_hst + @MITd...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|    false|     true|1322271849028456448|                 HST|        mit_hst|           2449|       Cambridge, MA|None|   null|
|Just when I am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|    false|     true|132

In [9]:
df.printSchema()

root
 |-- tweet_text: string (nullable = true)
 |-- hash_tag: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- retweeted: boolean (nullable = true)
 |-- truncated: boolean (nullable = true)
 |-- id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- followers_count: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- invalid: string (nullable = true)



In [10]:
df.take(1)

[Row(tweet_text='.@mit_hst + @MITdeptofBE prof. James Collins spoke about harnessing synthetic #Biology to develop diagnostics for #COVID19 + how his lab is using #DeepLearning  to enhance the design of such systems  @MITEECS #jclinic @MIT_IMES #MachineLearning https://t.co/2u3JwNj7CR https://t.co/EoAMQo7UHx', hash_tag='Biology, COVID19, DeepLearning, jclinic, MachineLearning', created_at='Fri Oct 30 20:19:10 +0000 2020', retweet_count=0, favorite_count=0, retweeted=False, truncated=True, id='1322271849028456448', user_name='HST', screen_name='mit_hst', followers_count=2449, location='Cambridge, MA', geo='None', invalid=None)]

In [14]:
df.count()

35463

In [11]:
print(file_schema.fieldNames())

['tweet_text', 'hash_tag', 'created_at', 'retweet_count', 'favorite_count', 'retweeted', 'truncated', 'id', 'user_name', 'screen_name', 'followers_count', 'location', 'geo', 'invalid']


In [12]:
# columns of interest
cols_select = ['tweet_text', 'hash_tag', 'created_at', 'retweet_count', 'favorite_count']

In [14]:
df_select = df.dropna(subset=["tweet_text"]).select(cols_select)
df_select.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|
+--------------------+--------------------+--------------------+-------------+--------------+
|.@mit_hst + @MITd...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|
|Just when I am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|
|And a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|
|Check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|
|South Korea 🇰🇷 ...|Healthcare, IoT, ...|Fri Oct 30 20:19:...|            0|             0|
+--------------------+--------------------+--------------------+-------------+--------------+
only showing top 5 rows



In [15]:
df_select.cache()
df_select.count()

35461

In [44]:
# df_select.unpersist()

DataFrame[tweet_text: string, hash_tag: string, created_at: string, retweet_count: int, favorite_count: int]

### 2. Text Processing

#### 2.1. Basic cleaning
1. remove mention @xxx, hashtag #xxx;
2. remove hyperlink;
3. remove special characters;
4. remove any whitespace characters (change it to one)

TODO: </br>
* also fix the abbreviation

In [16]:
df_select_clean = (df_select.withColumn("tweet_text", F.regexp_replace("tweet_text", r"[@#&][A-Za-z0-9_-]+", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"\w+:\/\/\S+", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"[^0-9A-Za-z \t]", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"\s+", " "))
                   .withColumn("tweet_text", F.lower(F.col("tweet_text")))
                   .withColumn("tweet_text", F.trim(F.col("tweet_text")))
                  )

In [17]:
df_select_clean.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|
+--------------------+--------------------+--------------------+-------------+--------------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|
|check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|
|south korea bus s...|Healthcare, IoT, ...|Fri Oct 30 20:19:...|            0|             0|
+--------------------+--------------------+--------------------+-------------+--------------+
only showing top 5 rows



In [27]:
df_select_clean.cache()

DataFrame[tweet_text: string, hash_tag: string, created_at: string, retweet_count: int, favorite_count: int]

In [18]:
df_sample_pd_unclean = df_select.limit(10).toPandas()

for i in range(10):
    print(df_sample_pd_unclean.iloc[i, 0], "\n")

.@mit_hst + @MITdeptofBE prof. James Collins spoke about harnessing synthetic #Biology to develop diagnostics for #COVID19 + how his lab is using #DeepLearning  to enhance the design of such systems  @MITEECS #jclinic @MIT_IMES #MachineLearning https://t.co/2u3JwNj7CR https://t.co/EoAMQo7UHx 

Just when I am about to have a glass of Gin and lemonade.... #Past3amSquad #AI #MachineLearning #DeepLearning #Coding https://t.co/sAOiSemArA 

And a beneficial use of #AI: Using Artificial Intelligence to Identify Powerful New Antibiotic #FridayReads  https://t.co/K8mPDdPShd 

Check out new work on my @Behance profile: "Community Bank / CREDIT CARD DESIGN" https://t.co/xAF0gSaniv   #AI 

South Korea 🇰🇷 bus 🚌 stops check your temperature 🤒 to fight coronavirus 😷 @wef   #Healthcare #IoT #AI #DigitalTransformation #Technology #Innovation #HealthTech via @automeme #SmartCity #COVID19 cc @mvollmer1 @IrmaRaste @Ronald_vanLoon @ipfconline1  https://t.co/YyW9hmONVX 

ARRIA is the language of data &amp; 

In [19]:
df_sample_pd_clean = df_select_clean.limit(10).toPandas()

In [20]:
for i in range(10):
    print(df_sample_pd_clean.iloc[i, 0], "\n")

prof james collins spoke about harnessing synthetic to develop diagnostics for how his lab is using to enhance the design of such systems 

just when i am about to have a glass of gin and lemonade 

and a beneficial use of using artificial intelligence to identify powerful new antibiotic 

check out new work on my profile community bank credit card design 

south korea bus stops check your temperature to fight coronavirus via cc 

arria is the language of data voice reports reports from join the leaders love always defeats fear 

the latest thinking different daily thanks to 

thank you for featuring our work in beautifully written fourier neural operator gets 1000x speedup on navier stokes pdes as a bonus also talks about time 

reminder massachusetts paid family and medical leave next steps 

labor employment e note october 2020 



In [23]:
df_select_clean.dropna(subset=["tweet_text"]).count()

35461

#### 2.2. Tokenize

In [22]:
from pyspark.ml.feature import Tokenizer

In [24]:
tokenzier = Tokenizer(inputCol="tweet_text", outputCol="tokens")

df_tokenized = tokenzier.transform(df_select_clean)

In [26]:
df_tokenized.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+--------------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|              tokens|
+--------------------+--------------------+--------------------+-------------+--------------+--------------------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|[prof, james, col...|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|[just, when, i, a...|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|[and, a, benefici...|
|check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|[check, out, new,...|
|south korea bus s...|Healthcare, IoT, ...|Fri Oct 30 20:19:...|            0|             0|[south, korea, bu...|
+--------------------+--------------------+--------------------+-------------+--

**some quick analyses**

In [30]:
df_select_clean_analysis = df_tokenized.withColumn("wordCounts", F.size(F.col("tokens")))
df_select_clean_analysis.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+--------------------+----------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|              tokens|wordCounts|
+--------------------+--------------------+--------------------+-------------+--------------+--------------------+----------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|[prof, james, col...|        23|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|[just, when, i, a...|        13|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|[and, a, benefici...|        13|
|check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|[check, out, new,...|        12|
|south korea bus s...|Healthcare, IoT, ...|Fri Oct 30 20:19:...|            0|             0|[south, korea, bu...|    

In [40]:
agg_col = ["wordCounts", "retweet_count", "favorite_count"]
# [F.min(F.col(c)).alias(c+"_min") for c in agg_col]
df_select_clean_analysis.agg(*[F.mean(F.col(c)).alias(c+"_mean") for c in agg_col],
                             *[F.min(F.col(c)).alias(c+"_min") for c in agg_col], 
                             *[F.max(F.col(c)).alias(c+"_max") for c in agg_col]).show()

+-----------------+------------------+-------------------+--------------+-----------------+------------------+--------------+-----------------+------------------+
|  wordCounts_mean|retweet_count_mean|favorite_count_mean|wordCounts_min|retweet_count_min|favorite_count_min|wordCounts_max|retweet_count_max|favorite_count_max|
+-----------------+------------------+-------------------+--------------+-----------------+------------------+--------------+-----------------+------------------+
|14.99416260116748|               0.0|                0.0|             1|                0|                 0|            61|                0|                 0|
+-----------------+------------------+-------------------+--------------+-----------------+------------------+--------------+-----------------+------------------+



In [39]:
df_select_clean_analysis.select(F.avg(F.col("wordCounts")), 
                                F.min(F.col("wordCounts")), 
                                F.max(F.col("wordCounts"))).show()

+-----------------+---------------+---------------+
|  avg(wordCounts)|min(wordCounts)|max(wordCounts)|
+-----------------+---------------+---------------+
|14.99416260116748|              1|             61|
+-----------------+---------------+---------------+



In [48]:
df_select_clean_analysis.approxQuantile(col="wordCounts", 
                                        probabilities=[0.25, 0.5, 0.75], 
                                        relativeError=0.1)

[8.0, 13.0, 20.0]

#### 2.3. Remove stop words
TODO: extend stop words </br>
("also", "via", "cc", ...)

In [49]:
from pyspark.ml.feature import StopWordsRemover

In [50]:
stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="remove_stop")

In [54]:
print(sorted(stopword_remover.getStopWords()))

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', 'just', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'now', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 's', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that's", 'the', 'the

In [56]:
df_rmstop = stopword_remover.transform(df_tokenized)

In [57]:
df_rmstop.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+--------------------+--------------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|              tokens|         remove_stop|
+--------------------+--------------------+--------------------+-------------+--------------+--------------------+--------------------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|[prof, james, col...|[prof, james, col...|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|[just, when, i, a...|[glass, gin, lemo...|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|[and, a, benefici...|[beneficial, use,...|
|check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|[check, out, new,...|[check, new, work...|
|south korea bus s...|Healthcare, IoT, ...|Fri O

In [58]:
# df_sample_pd_before = df_tokenized.limit(10).toPandas()
df_sample_pd_rmstop = df_rmstop.limit(10).toPandas()

for i in range(10):
    print(df_sample_pd_rmstop.loc[i, "tokens"], "\n")
    print(df_sample_pd_rmstop.loc[i, "remove_stop"], "\n")

['prof', 'james', 'collins', 'spoke', 'about', 'harnessing', 'synthetic', 'to', 'develop', 'diagnostics', 'for', 'how', 'his', 'lab', 'is', 'using', 'to', 'enhance', 'the', 'design', 'of', 'such', 'systems'] 

['prof', 'james', 'collins', 'spoke', 'harnessing', 'synthetic', 'develop', 'diagnostics', 'lab', 'using', 'enhance', 'design', 'systems'] 

['just', 'when', 'i', 'am', 'about', 'to', 'have', 'a', 'glass', 'of', 'gin', 'and', 'lemonade'] 

['glass', 'gin', 'lemonade'] 

['and', 'a', 'beneficial', 'use', 'of', 'using', 'artificial', 'intelligence', 'to', 'identify', 'powerful', 'new', 'antibiotic'] 

['beneficial', 'use', 'using', 'artificial', 'intelligence', 'identify', 'powerful', 'new', 'antibiotic'] 

['check', 'out', 'new', 'work', 'on', 'my', 'profile', 'community', 'bank', 'credit', 'card', 'design'] 

['check', 'new', 'work', 'profile', 'community', 'bank', 'credit', 'card', 'design'] 

['south', 'korea', 'bus', 'stops', 'check', 'your', 'temperature', 'to', 'fight', 'cor

#### 2.4. Lemmatization

In [68]:
df_rmstop.printSchema()

root
 |-- tweet_text: string (nullable = true)
 |-- hash_tag: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- remove_stop: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [59]:
# Import stemmer library
from nltk.stem.porter import PorterStemmer

In [60]:
# Instantiate stemmer object
stemmer = PorterStemmer()

**so pandas_udf cannot handle complex data?**

In [69]:
def stem(words_list):
    return [stemmer.stem(word) for word in words_list]

stem_udf = F.udf(stem, returnType = ArrayType(StringType()))

In [71]:
df_stemmed = df_rmstop.select("remove_stop", stem_udf(F.col("remove_stop")).alias("stemmed")).limit(10).toPandas()

In [72]:
for i in range(10):
    print(df_stemmed.loc[i, "remove_stop"], "\n")
    print(df_stemmed.loc[i, "stemmed"], "\n")

['prof', 'james', 'collins', 'spoke', 'harnessing', 'synthetic', 'develop', 'diagnostics', 'lab', 'using', 'enhance', 'design', 'systems'] 

['prof', 'jame', 'collin', 'spoke', 'har', 'synthet', 'develop', 'diagnost', 'lab', 'use', 'enhanc', 'design', 'system'] 

['glass', 'gin', 'lemonade'] 

['glass', 'gin', 'lemonad'] 

['beneficial', 'use', 'using', 'artificial', 'intelligence', 'identify', 'powerful', 'new', 'antibiotic'] 

['benefici', 'use', 'use', 'artifici', 'intellig', 'identifi', 'power', 'new', 'antibiot'] 

['check', 'new', 'work', 'profile', 'community', 'bank', 'credit', 'card', 'design'] 

['check', 'new', 'work', 'profil', 'commun', 'bank', 'credit', 'card', 'design'] 

['south', 'korea', 'bus', 'stops', 'check', 'temperature', 'fight', 'coronavirus', 'via', 'cc'] 

['south', 'korea', 'bu', 'stop', 'check', 'temperatur', 'fight', 'coronaviru', 'via', 'cc'] 

['arria', 'language', 'data', 'voice', 'reports', 'reports', 'join', 'leaders', 'love', 'always', 'defeats', 'fe

In [73]:
df_stemmed = df_rmstop.withColumn("stemmed", stem_udf(F.col("remove_stop")))

df_stemmed.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+--------------------+--------------------+--------------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|              tokens|         remove_stop|             stemmed|
+--------------------+--------------------+--------------------+-------------+--------------+--------------------+--------------------+--------------------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|[prof, james, col...|[prof, james, col...|[prof, jame, coll...|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|[just, when, i, a...|[glass, gin, lemo...|[glass, gin, lemo...|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|[and, a, benefici...|[beneficial, use,...|[benefici, use, u...|
|check out new wor...|                  AI|Fri Oct 30 20:1

In [74]:
df_stemmed.cache()
df_stemmed.count()

35461

#### 2.5. More to consider
1. use bigram
2. filter short vector

#### 2.6. Encode words

In [82]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF

##### 2.6.1. CountVectorizer

In [77]:
# should tune some parameter
vectorizer = CountVectorizer(inputCol= "stemmed", outputCol="rawFeatures")
vectorizer_model = vectorizer.fit(df_stemmed)

In [78]:
df_vect = vectorizer_model.transform(df_stemmed)

In [80]:
df_vect.select("stemmed", "rawFeatures").show(5, truncate=False)

+----------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|stemmed                                                                                       |rawFeatures                                                                                                          |
+----------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|[prof, jame, collin, spoke, har, synthet, develop, diagnost, lab, use, enhanc, design, system]|(11920,[4,20,42,132,385,665,956,1203,1371,1681,1683,2374,4531],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|[glass, gin, lemonad]                                                                         |(11920,[2418,6542,6640],[1.0,1.0,1.0])      

##### 2.6.2. TF-IDF
**do we need IDF for the LDA model?**

In [83]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(df_vect)

In [85]:
df_idf = idf_model.transform(df_vect)

In [86]:
df_idf.select("rawFeatures", "features").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|rawFeatures                                                                                                          |features                                                                                                                                                                                                                                                                                                    |
+---------------------------------------------------------------------------------------------------------------------+---------------------------------------

#### 2.7. Fit a LDA model

In [87]:
from pyspark.ml.clustering import LDA

In [89]:
lda = LDA(k=10, seed=1, optimizer="em")

In [90]:
lda_mode = lda.fit(df_idf)

In [91]:
type(lda_mode)

pyspark.ml.clustering.DistributedLDAModel

In [107]:
topics_describe = lda_mode.describeTopics()
topics_describe.show(truncate=False)

+-----+---------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termIndices                                  |termWeights                                                                                                                                                                                                                   |
+-----+---------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[4, 71, 102, 27, 10, 362, 260, 182, 2, 372]  |[0.009657542214926401, 0.007927589170209906, 0.007051729791972844, 0.006599549862990243, 0.006240483387209483, 0.006

##### 2.7.1. convert the word vector to vocabulary

In [102]:
vectorizer_model.getVocabSize()

262144

In [103]:
vectorizer_model.vocabulary[0:10]

['learn',
 'ai',
 'data',
 'via',
 'use',
 'intellig',
 'machin',
 'artifici',
 'new',
 'cc']

In [109]:
def termsIdx2Term(vocabulary):
    def termsIdx2Term(termIndices):
        return [vocabulary[int(index)] for index in termIndices]
    return F.udf(termsIdx2Term, ArrayType(StringType()))

vocabList = vectorizer_model.vocabulary

In [110]:
final = topics_describe.withColumn("Terms", termsIdx2Term(vocabList)("termIndices"))

In [111]:
final.select("Terms").show(truncate=False)

+--------------------------------------------------------------------------+
|Terms                                                                     |
+--------------------------------------------------------------------------+
|[use, great, train, best, read, 000, high, care, data, ve]                |
|[10, excit, 100, next, year, look, data, learn, re, novemb]               |
|[import, technolog, world, ai, data, learn, see, artifici, best, machin]  |
|[learn, cours, first, 1, python, mt, ai, world, creat, complet]           |
|[compani, secur, financi, futur, system, process, effici, learn, cc, must]|
|[role, ai, base, drive, busi, transform, use, program, process, product]  |
|[top, predict, learn, check, via, v, websit, student, challeng, 2]        |
|[report, leader, detect, languag, love, record, covid, learn, alway, imag]|
|[rt, code, book, intellig, provid, smart, innov, analyt, valu, 50]        |
|[free, tool, ai, develop, network, amaz, neural, color, avail, learn]     |

##### 2.7.2. check the model performance
**Note:** the calculation of log Perplexity requires a large topicsMatrix() to the driver. So one should use subsample...

In [96]:
lda_mode.logPerplexity(df_idf)

11.142064829812528

In [97]:
df_sample_check_score = df_idf.sample(fraction=0.2, seed=1234)

In [98]:
lda_mode.logPerplexity(df_sample_check_score)

24.98677336119014